In [1]:
import pandas as pd
import os
import glob
import numpy as np
import smartsheet

#Variables
# random
excel = '.xlsx'
csv = '.csv'

# smartsheets Shet IDs
Central_id = '1844440261257092'
South_id= '6292262490531716'

# Central's API
Central = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') #API

Central.errors_as_exceptions(True)

# Central's API
South = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API
South.errors_as_exceptions(True)

#download smartsheets to any folder you need
# Central.Sheets.get_sheet_as_excel(Central_id, download_path=r"C:\Users\jacque.trahan\Downloads")
# South.Sheets.get_sheet_as_excel(South_id, download_path=r"C:\Users\jacque.trahan\Downloads") 


#Google Sheets
#Sheet ID comes right after /d/ in the URL
Google_sheet_ID= '1O2dmdhEo5hJVAgRVDl4M4n3_v-KqHS56q3OZVdpZANU'
# name of tab you want to use
Google_sheets_name = 'Tree RTR Tracker'

# this is tricky. after /export? we must put format=csv#id={Sheet Name}   -> manually to make this work
url = f'https://docs.google.com/spreadsheets/d/{Google_sheet_ID}/export?format=csv#id={Google_sheets_name}'



# Function to read in column titles and row vales and make out data frames from smartsheets
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Central_SS = smartsheet_loadup(Central.Sheets.get_sheet(Central_id))
South_SS = smartsheet_loadup(South.Sheets.get_sheet(South_id))

df_both = pd.concat([Central_SS,South_SS])


# Smartsheets now uploaded by API below method no longer used

In [2]:
# path1 = r"C:\Users\jacque.trahan\Downloads\Central Division 2021 Debris Removal Tracker*"
# file_type1 = ".xlsx"
# load1 = glob.glob(path1 + file_type1)
# df = pd.read_excel(max(load1, key=os.path.getctime))

In [3]:
# path2 = r"C:\Users\jacque.trahan\Downloads\South Central Division 2021 Debris Removal Tracker*"
# load2 = glob.glob(path2 + file_type1)
# df2 = pd.read_excel(max(load2, key=os.path.getctime))
# df2.rename(columns=({'Propane Tanks': 'Propane Tank'}),inplace=True)

In [4]:
# df_both = pd.concat([df, df2])

In [5]:
# df_both.info(verbose=True)

In [6]:
# check = df_both[['APN_ROW Segment', 'Structural Status', 'Street #', 'Street Name','County', 'Debris Crew','ROE Type',
#                  'Haz Tree Status','Tree Felling Start', 'Tree Felling Complete','Trees Completion Check',
#                  'Soil Samples Approved',
#                  'Erosion Control Complete','Erosion Control Waiver']]

In [7]:
# check.to_excel('Check FSW and Debris Crew.xlsx')

In [8]:
FSO = df_both.copy()
FSO = FSO[['APN_ROW Segment', 'Structural Status', 'Street #', 'Street Name','County',
                  'Haz Tree Status',"Tree Removal Complete", # should say TFSW complete willneed to check this
                  'Soil Samples Approved',
                  'Erosion Control Complete', 'Erosion Control Waiver',
                  "USFS Parcel?", "Needs Milk Run or Phase 1 Revisit",
                   "811 Called (A&M)", "811 Called (DDHTR)",
                   "Vehicle(s)", 'Number of Vehicles Removed',
                   "ROE Date",
                   'FSO Complete', 'FSO Sent to County','Debris Crew',
           'Number of Trees','Tree Removal Start', 'Tree Removal Complete',
           'Tree Felling Start', 'Tree Felling Complete',
           'Number of Hazard Trees Felled','Trees Completion Check', 'ROE Type', 'Tree Crew', 'Trees Completion Check',
           'Total Number of Vehicles',
           'Hold', 'Hold Reason', 'Contact Notes', 'Propane Tank', 'Propane Tank Removal', 'Date Propane Tank Removed',
           'Nesting Bird Survey Date', 'City'
       ]]

FSO["Vehicle(s)"] = np.where(FSO["Vehicle(s)"] == 1, "True", "")
FSO["ROE Date"] = FSO["ROE Date"].fillna("")
FSO.set_index('APN_ROW Segment', inplace=True)
FSO.to_excel("Smartsheets data.xlsx")
FSO.reset_index(inplace=True)

# Read in and set up RTR Data

In [9]:
# path2 = r"C:\Users\jacque.trahan\Downloads\RTR Data Live*"
# file_type2 = ".xlsx"
# load2 = glob.glob(path2 + file_type2)
# df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True)

In [10]:
# df2.info()

In [11]:
path2 = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
file_type2 = ".xlsx"
load2 = glob.glob(path2 + file_type2)
df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True)

In [12]:
RTR2 = df2.copy()
RTR2['Zone Name'] = RTR2['Zone Name'].str.strip()
RTR2['Zone Name'] = RTR2['Zone Name'].replace("-", "")
RTR2['Zone Name'] = RTR2['Zone Name'].str[:3] + "-" + RTR2['Zone Name'].str[3:6] + "-" + RTR2['Zone Name'].str[6:]
RTR2['Service Code'] = RTR2['Service Code'].str.strip()

# RTR Tree

In [13]:
RTR = RTR2.copy()
#set up columns & conditions for trees
RTR = RTR[['Zone Name', 'Service Code', 'Unit Count', 'Is Void', 'GIS Zone1','GIS Zone2','Start Time', 'End Time',"Addr No", "Addr St", "Ticket Number"]]
RTR = RTR[(RTR['Is Void'] == False) & (RTR['Service Code'].isin(['12A', '12B', '76', '55A','55B'])) & ~(RTR['GIS Zone1'].isin(['NORTHERN DIVISION']))] 

RTR.set_index('Zone Name', inplace = True)
RTR.to_excel('RTR Data for Trees.xlsx')

# RTR Data For Erosion

In [14]:
RTR_erosion = RTR2.copy()

#Set up columns
RTR_erosion = RTR_erosion[['Zone Name', 'Service Code', 'Unit Count', 'Is Void', 'GIS Zone1','GIS Zone2', 'Ticket Date',
             "Addr No", "Addr St", "Ticket Number", "End Time", "Disposal Monitor Name"]]

#set up conditions for erosion
RTR_erosion = RTR_erosion[(RTR_erosion['Is Void'] == False) & (RTR_erosion['Service Code'] == '73A')] 
RTR_erosion =  RTR_erosion[~(RTR_erosion['GIS Zone1'].isin(['NORTHERN DIVISION']))]

# format dates without times
RTR_erosion['Ticket Date'] = pd.to_datetime(RTR_erosion['Ticket Date']).dt.date

# set zone name as index and make an excel sheet
RTR_erosion.set_index('Zone Name', inplace=True)
RTR_erosion.to_excel('RTR Data For Erosion.xlsx')

# RTR_erosion = RTR_erosion.add_suffix('_RTR')
# RTR_erosion.reset_index(inplace=True)

# Propane tank removal - Service code is 2B

In [15]:
Propane = RTR2.copy()

Propane = Propane[['Zone Name', 'Service Code', 'Unit Count', 'Is Void', 'GIS Zone1','GIS Zone2', 'Ticket Date',
             "Addr No", "Addr St", "Ticket Number","End Time", "Disposal Monitor Name"]]
Propane = Propane[(Propane['Service Code'].isin(['2B'])) & (Propane['Is Void'] == False) & ~(Propane['GIS Zone1'].isin(['NORTHERN DIVISION']))]
Propane['Ticket Date'] = pd.to_datetime(Propane['Ticket Date']).dt.date


Propane.set_index('Zone Name', inplace=True)
Propane.to_excel('RTR Data for Propane Tanks.xlsx')

# Check Propane tanks missing

In [16]:
propane = df_both.copy()

propane = propane[['APN_ROW Segment', 'Structural Status', 'Street #', 'Street Name','County',
 'Propane Tank', 'Propane Tank Removal', 'Date Propane Tank Removed']]

propane = propane[(propane['Propane Tank'].isnull())]
propane.to_excel("Missing Propane Status.xlsx")